In [1]:
from datetime import datetime
import pandas as pd

import requests
import time

In [2]:
# API setup

key = '...'
url = 'http://api.weatherapi.com/v1/history.json'

In [3]:
# REQUEST Info Here

gps_coord = (45.7170348,5.0091914)

start_date = '2021-01-03'
end_date = '2021-12-26'

In [4]:
# We first divide the range into FULL MONTH RANGES

# Date : conversion to datetime
start_datetime = datetime.strptime(start_date,'%Y-%m-%d')
start_datetime = start_datetime.replace(day=1)
end_datetime = datetime.strptime(end_date,'%Y-%m-%d')

# We create the start date and stop date of each month
starts = pd.date_range(start=start_datetime,end=end_datetime,freq='MS')
ends = pd.date_range(start=starts[0],periods=len(starts),freq='M')

# We convert them to strings
starts = [ d.strftime('%Y-%m-%d') for d in starts ]
ends = [ d.strftime('%Y-%m-%d') for d in ends ]

# We replace the first and last date
starts[0] = start_date
ends[-1] = end_date

months = tuple(zip(starts,ends))


In [5]:
# API REQUEST

# We'll store the infos in that list
extracted_meteo = []

for month in months :
    
    print(f'Requesting {month[0]}...',end='')
    
    dt = month[0]                                      # API : start date of the request
    end_dt = month[1]                                  # API : stop date
    q = str(gps_coord[0]) + ',' + str(gps_coord[1])    # API : GPS Coordinates formatted
    
    params = { 'key':key, 'q':q, 'dt':dt, 'end_dt':end_dt }
    
    response = requests.get(url,params=params)
    
    # if code response != 4XX, we wait and retry one time
    # (sometimes, the API doesn't respond)
    if not response : 
        time.sleep(2)
        response = requests.get(url,params=params)
        
    jr = response.json()
    forecast_list = jr.get('forecast').get('forecastday')
    
    for forecast in forecast_list :
    
        date = forecast['date']
        temp = forecast['day']['avgtemp_c']
        precipitation = forecast['day']['totalprecip_mm']
        maxwind = forecast['day']['maxwind_kph']
        condition = forecast['day']['condition']['text']

        extracted_meteo.append((date,temp,precipitation,maxwind,condition))
    
    print('ok')
    

Requesting 2021-01-03...ok
Requesting 2021-02-01...ok
Requesting 2021-03-01...ok
Requesting 2021-04-01...ok
Requesting 2021-05-01...ok
Requesting 2021-06-01...ok
Requesting 2021-07-01...ok
Requesting 2021-08-01...ok
Requesting 2021-09-01...ok
Requesting 2021-10-01...ok
Requesting 2021-11-01...ok
Requesting 2021-12-01...ok


In [6]:
# Conversion to a Dataframe
columns = [
    'date',
    'temperature',    # °C
    'precipitation',  # mm
    'maxwind',        # km/h
    'condition'       # description
]

meteo_df = pd.DataFrame(extracted_meteo,columns=columns)
meteo_df.set_index('date',inplace=True)

In [7]:
meteo_df

,temperature,precipitation,maxwind,condition
date,,,,
2021-01-03,0.8,0.8,18.0,Moderate snow
2021-01-04,0.4,0.0,9.7,Light snow
2021-01-05,0.6,0.5,8.3,Moderate snow
2021-01-06,0.5,0.5,7.9,Moderate snow
2021-01-07,0.7,0.4,10.1,Moderate snow
...,...,...,...,...
2021-12-22,2.8,0.0,12.6,Sunny
2021-12-23,4.9,0.0,14.4,Overcast
2021-12-24,6.7,0.0,12.6,Cloudy
